In [1]:
# Use non-inline plotting for visualization.
%matplotlib

Using matplotlib backend: TkAgg


In [2]:
import math
import numpy as np

from pydrake.all import (
    DiagramBuilder,
    FindResourceOrThrow,
    FloatingBaseType,
    BasicVector,
    LinearQuadraticRegulator,
    Simulator,
    RigidBodyTree,
    RigidBodyPlant,
)
from underactuated import (
    PlanarRigidBodyVisualizer,
)

In [3]:
builder = DiagramBuilder()
tree = RigidBodyTree(FindResourceOrThrow("drake/examples/pendulum/Pendulum.urdf"),
                     FloatingBaseType.kFixed)

robot = builder.AddSystem(RigidBodyPlant(tree))

def UprightState():
    state = (math.pi, 0)
    return state
def BalancingLQR(robot):
    # Design an LQR controller for stabilizing the CartPole around the upright.
    # Returns a (static) AffineSystem that implements the controller (in
    # the original CartPole coordinates).

    context = robot.CreateDefaultContext()
    context.FixInputPort(0, BasicVector([0]))
    context.get_mutable_continuous_state_vector().SetFromVector(UprightState())

    Q = np.diag((10., 1.))
    R = [1]

    return LinearQuadraticRegulator(robot, context, Q, R)


controller = builder.AddSystem(BalancingLQR(robot))


builder.Connect(robot.get_output_port(0), controller.get_input_port(0))
builder.Connect(controller.get_output_port(0), robot.get_input_port(0))


visualizer = builder.AddSystem(PlanarRigidBodyVisualizer(tree, xlim=[-2.5, 2.5], ylim=[-1, 2.5]))
builder.Connect(robot.get_output_port(0), visualizer.get_input_port(0))

diagram = builder.Build()
simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)
simulator.set_publish_every_time_step(False)

context = simulator.get_mutable_context()
state = context.get_mutable_continuous_state_vector()

Spawning PlanarRigidBodyVisualizer for tree with 1 actuators


In [4]:
duration = 1.
simulator.StepTo(duration)